In [17]:
import pandas as pd
import numpy as np
from astropy.coordinates import SkyCoord
from astropy import units as u

In [20]:
df = pd.read_csv(
    '/home/rt2122/Data/scans_extended/connected/full_act_cut_ep10_thr0.1_step8.csv')

In [5]:
cats = [cat for cat in set(df['catalog']) if type(cat) == type('')]
cats

['act_erosita', 'planck_no_z', 'planck_z', 'mcxcwp']

In [12]:
for cat in cats:
    df_cur = df[df['catalog'] == cat]
    print(cat, 'tp', np.count_nonzero(df_cur['status'] == 'tp'), 
          'fn', np.count_nonzero(df_cur['status'] == 'fn'))

act_erosita tp 0 fn 0
planck_no_z tp 364 fn 195
planck_z tp 991 fn 103
mcxcwp tp 158 fn 1035


In [9]:
idx = df['catalog'] == 'act_erosita'
np.count_nonzero(idx)

2980

In [11]:
df['catalog'].loc[idx] = np.nan

/home/rt2122/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [14]:
df['status'].loc[idx] = 'fn'

In [15]:
act = pd.read_csv('/home/rt2122/Data/clusters/act.csv')

In [16]:
df_fp = df[df['status'] == 'fp']
df_fp.index = np.arange(len(df_fp))
df_tpfn = df[df['status'] != 'fp']
df_tpfn.index = np.arange(len(df_tpfn))

In [19]:
fp_sc = SkyCoord(ra=np.array(df_fp['RA'])*u.degree, 
                 dec=np.array(df_fp['DEC'])*u.degree, frame='icrs')
act_sc = SkyCoord(ra=np.array(act['RA'])*u.degree, 
                  dec=np.array(act['DEC'])*u.degree, frame='icrs')

In [ ]:
idx, d2d, _ = fp_sc.match_to_catalog_sky(act_sc)
matched = d2d.degree <= 5/60

#res_cat['status'] = 'fp'
df_fn['status'].iloc[matched] = 'tp'
df_fn['catalog'].iloc[matched] = 'act' 
df_fn['tRA'].iloc[matched] = np.array(act['RA'][idx[matched]])
df_fn['tDEC'].iloc[matched] = np.array(act['DEC'][idx[matched]])

df_fn = df_fn.drop_duplicates(subset=['tRA', 'tDEC'])

act['found'] = False
act['found'].iloc[idx[matched]] = True
act['status'] = 'fn'
